In [1]:
using Pkg
Pkg.activate("C:\\Users\\zachl\\OneDrive\\Documents\\GitHub\\StateSpaceDynamics.jl\\benchmarking")
Pkg.instantiate()

  Activating project at `C:\Users\zachl\OneDrive\Documents\GitHub\StateSpaceDynamics.jl\benchmarking`


In [1]:
using BenchmarkTools
using Random
using Statistics
# using PythonCall
using DataFrames
using Printf
using LinearAlgebra
using CSV
using Distributions
using HiddenMarkovModels
using StateSpaceDynamics

In [3]:
# Create true model
true_model, hmmjl = generate_random_hmm(2, 2)

# Generate test data
true_model, true_labels, data = generate_test_data(true_model, 1000)

UndefVarError: UndefVarError: `generate_random_hmm` not defined

In [250]:
println(true_model.B[1])
println(true_model.B[2])
println(true_model.A)

GaussianEmission(2, [-0.08154554544971672, 0.7737722708677196], [3.54308849520104 1.5433570866323696; 1.5433570866323696 2.00352302061728])
GaussianEmission(2, [-0.35389974086822495, -1.0362061789816457], [6.333621577291121 -1.7799757090298411; -1.7799757090298411 5.373991524626046])
[0.9209880708086438 0.07901192919135626; 0.2682645423197521 0.731735457680248]


In [6]:
using StateSpaceDynamics

In [11]:
# Practice fitting hidden markov models .jl so i know how to write the Benchmarking
init = [0.6, 0.4]
trans = [0.7 0.3; 0.2 0.8]
dists = [MvNormal([-0.5, -0.8], I), MvNormal([0.5, 0.8], I)]
hmm = HMM(init, trans, dists)

T = 10000
state_seq, obs_seq = rand(hmm, T);

A = rand()
Σ = Matrix(A.*I(2))

init_guess = [0.5, 0.5]
trans_guess = [0.6 0.4; 0.3 0.7]
dists_guess = [MvNormal([-0.4, -0.7], Σ), MvNormal([0.4, 0.7], Σ)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

vectorized_data = [data[:, i] for i in 1:size(data, 2)]
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, vectorized_data, atol=1e-10, max_iterations=100);

UndefVarError: UndefVarError: `data` not defined

In [238]:
hmm_guess

Hidden Markov Model with:
 - initialization: [0.5, 0.5]
 - transition matrix: [0.6 0.4; 0.3 0.7]
 - observation distributions: [IsoNormal(
dim: 2
μ: [-0.4, -0.7]
Σ: [0.8 0.0; 0.0 0.8]
)
, FullNormal(
dim: 2
μ: [0.4, 0.7]
Σ: [0.1 0.2; 0.2 1.0]
)
]

In [2]:
using StateSpaceDynamics
# using Distributions
using LinearAlgebra

In [3]:
# Create Guassian Emission Models
output_dim = 2
μ = [-0.4, -0.7]
Σ = 1 * Matrix{Float64}(I, output_dim, output_dim)
emission_1 = GaussianEmission(output_dim, μ, Σ)

μ = [0.4, 0.7]
Σ = 1 * Matrix{Float64}(I, output_dim, output_dim)
emission_2 = GaussianEmission(output_dim, μ, Σ)

# Create GaussianHMM
test_model = StateSpaceDynamics.GaussianHMM(K=2, output_dim=2)
test_model.B[1] = emission_1
test_model.B[2] = emission_2

# Fit a gaussian hmm to the data
test_model = StateSpaceDynamics.GaussianHMM(K=2, output_dim=2)
# ll = StateSpaceDynamics.fit!(test_model, data, max_iters=100)

HiddenMarkovModel([0.19616886057506838 0.8038311394249317; 0.5853816187992231 0.4146183812007768], EmissionModel[GaussianEmission(2, [0.0, 0.0], [1.0 0.0; 0.0 1.0]), GaussianEmission(2, [0.0, 0.0], [1.0 0.0; 0.0 1.0])], [0.23734426101889405, 0.7626557389811058], 2)

In [181]:
println("True A: ", true_model.A)
println("Our A: ", test_model.A)
println("HMMjl A: ", hmm_est.trans)
println()

println("True B1: ", true_model.B[1])
println("Our B1: ", test_model.B[1])
println("HMMjl B1: ", hmm_est.dists[1])
println()

println("True B2: ", true_model.B[2])
println("Our B2: ",test_model.B[2])
println("HMMjl B2: ", hmm_est.dists[2])


True A: [0.9860801028269849 0.01391989717301509; 0.3821770814044893 0.6178229185955108]
Our A: [0.8567766358605983 0.14322336413940706; 0.7376890394366028 0.2623109605633596]
HMMjl A: [0.46959980947035035 0.5304001905296496; 0.42600605952853393 0.5739939404714662]

True B1: GaussianEmission(2, [0.009552118837806297, -0.4360932926646945], [2.815031376770079 -0.5029464781416295; -0.5029464781416295 0.3840947405591081])
Our B1: GaussianEmission(2, [0.09633762011893998, -0.474131717682502], [3.0178950303320105 -0.5225241442338352; -0.5225241442338352 0.33328699344802504])
HMMjl B1: IsoNormal(
dim: 2
μ: [-1.2897983794670145, -0.20502239360039295]
Σ: [0.7383945937979036 0.0; 0.0 0.7383945937979036]
)


True B2: GaussianEmission(2, [1.61419748504034, 0.23338149213477438], [0.4803417151149 -0.5726288216791778; -0.5726288216791778 1.4628299994812846])
Our B2: GaussianEmission(2, [0.3099790297627449, -0.2714169785523996], [1.8419560046639865 -0.12100509266521373; -0.12100509266521373 0.893128897

In [362]:
dists_guess_orig = [MvNormal([-0.4, -0.7], Σ), MvNormal([0.4, 0.7], Σ)]
typeof(dists_guess_orig[1])

DiagNormal (alias for MvNormal{Float64, PDMats.PDiagMat{Float64, Array{Float64, 1}}, Array{Float64, 1}})

In [2]:
struct BenchConfig
    latent_dims::Vector{Int}
    obs_dims::Vector{Int} 
    seq_lengths::Vector{Int}
    n_iters::Int
    n_repeats::Int
end

default_config = BenchConfig(
    [2, 4, 8, 16],       # latent dimensions
    [4, 8, 16, 32],      # observation dimensions
    [100, 500, 1000],    # sequence lengths
    100,                 # EM iterations
    5                    # benchmark repeats
)

function initialize_transition_matrix(K::Int)
    # Initialize a transition matrix with zeros
    A = zeros(Float64, K, K)
    
    for i in 1:K
        # Sample from a Dirichlet distribution
        A[i, :] = rand(Dirichlet(ones(K)))
    end

    A .+= 0.5.*I(K)
    A .= A ./ sum(A, dims=2)
    return A
end


function initialize_state_distribution(K::Int)
    # initialize a state distribution
    return rand(Dirichlet(ones(K)))
end


function generate_random_hmm(latent_dim::Int, obs_dim::Int)
    # Create Gaussian Emission Models with random means and covariances
    emissions = Vector{GaussianEmission}(undef, latent_dim)
    dists_guess = Vector{MvNormal}(undef, latent_dim)  # Change to MvNormal

    for i in 1:latent_dim
        μ = randn(obs_dim)
        A = rand()
        Σ = I(obs_dim) # Diagonal covariance matrix, abs to ensure posdef
        emissions[i] = GaussianEmission(obs_dim, μ, Matrix(Σ))
        dists_guess[i] = MvNormal(μ, Matrix(Σ))  # Assign MvNormal with full covariance
    end

    # Create GaussianHMM
    true_model = StateSpaceDynamics.GaussianHMM(K=latent_dim, output_dim=obs_dim)
    
    for i in 1:latent_dim
        true_model.B[i] = emissions[i]
    end
    
    true_model.A = initialize_transition_matrix(latent_dim)
    true_model.πₖ = initialize_state_distribution(latent_dim)
    init_guess = true_model.πₖ
    trans_guess = true_model.A


    hmm_guess = HMM(init_guess, trans_guess, dists_guess);

    return true_model, hmm_guess
end


function generate_test_data(model, seq_len::Int)
    # Sample from the model
    labels, data = StateSpaceDynamics.sample(model, n=seq_len)

    return model, labels, data
end


function run_single_benchmark(model_type::Symbol, hmm_ssd, y, params=nothing; config=default_config)
    if model_type == :julia
        bench = @benchmark begin
            model = deepcopy($hmm_ssd)  # Create a fresh copy for each iteration
            StateSpaceDynamics.fit!(model, $y, max_iters=$config.n_iters, tol=1e-6)
        end samples=config.n_repeats
        return (time=minimum(bench).time, memory=bench.memory, allocs=bench.allocs, success=true)
    elseif model_type == :hmmjl
        bench = @benchmark begin
            model = deepcopy($hmm_ssd)
            baum_welch(model, $y, atol=1e-6, max_iterations=$config.n_iters, loglikelihood_increasing=false)
        end samples=config.n_repeats
        return (time=minimum(bench).time, memory=bench.memory, allocs=bench.allocs, success=true)
    end
end

function benchmark_fitting(config::BenchConfig = default_config)
    results = []

    for latent_dim in config.latent_dims
        for obs_dim in config.obs_dims
            obs_dim < latent_dim && continue

            for seq_len in config.seq_lengths
                println("\nTesting configuration: latent_dim=$latent_dim, obs_dim=$obs_dim, seq_len=$seq_len")

                # Create true model
                true_model, _ = generate_random_hmm(latent_dim, obs_dim)
                
                # Generate test data
                true_model, true_labels, data = generate_test_data(true_model, seq_len)
                vectorized_data = [data[:, i] for i in 1:size(data, 2)]  # Vectorize for HMMjl

                # Generate random HMMs for fitting
                test_model, HMMjl = generate_random_hmm(latent_dim, obs_dim)

                # Run benchmarks separately with error handling
                julia_result = try
                    run_single_benchmark(:julia, test_model, data)
                catch err
                    println("Error in SSD.jl benchmarking: ", err)
                    (time="FAIL", memory="FAIL", allocs="FAIL", success=false)
                end

                HMMjl_result = try
                    run_single_benchmark(:hmmjl, HMMjl, vectorized_data)
                catch err
                    println("Error in HMM.jl benchmarking: ", err)
                    (time="FAIL", memory="FAIL", allocs="FAIL", success=false)
                end

                # Save results
                push!(results, Dict(
                    "config" => (latent_dim=latent_dim, obs_dim=obs_dim, seq_len=seq_len),
                    "SSD.jl" => julia_result,
                    "HMM.jl" => HMMjl_result
                ))
            end
        end
    end

    return results
end

benchmark_fitting (generic function with 2 methods)

In [6]:
generate_random_hmm(1, 2)[1]

HiddenMarkovModel([1.0;;], EmissionModel[GaussianEmission(2, [0.9805306695039356, -0.7356050416988161], Bool[1 0; 0 1])], [0.9999999999999999], 1)

In [ ]:
results = benchmark_fitting()

In [4]:
using CSV
using DataFrames

function prepare_results_for_csv(results)
    rows = []
    for result in results
        config = result["config"]
        ssd = result["SSD.jl"]
        hmmjl = result["HMM.jl"]

        # Add a row for SSD.jl
        push!(rows, (
            latent_dim=config.latent_dim,
            obs_dim=config.obs_dim,
            seq_len=config.seq_len,
            library="SSD.jl",
            time=ssd.time,
            memory=ssd.memory,
            allocs=ssd.allocs,
            success=ssd.success,
        ))

        # Add a row for HMM.jl
        push!(rows, (
            latent_dim=config.latent_dim,
            obs_dim=config.obs_dim,
            seq_len=config.seq_len,
            library="HMM.jl",
            time=hmmjl.time,
            memory=hmmjl.memory,
            allocs=hmmjl.allocs,
            success=hmmjl.success,
        ))
    end
    return DataFrame(rows)
end

results_df = prepare_results_for_csv(results)
CSV.write("benchmark_results.csv", results_df)


"benchmark_results.csv"

In [17]:
# Create true model
latent_dim=2
obs_dim=8
seq_len=500

true_model, _ = generate_random_hmm(latent_dim, obs_dim)
                
# Generate test data
true_model, true_labels, data = generate_test_data(true_model, seq_len)
vectorized_data = [data[:, i] for i in 1:size(data, 2)]  # Vectorize for HMMjl

# Generate random HMMs for fitting
test_model, HMMjl = generate_random_hmm(latent_dim, obs_dim)

# Test the HMM.jl package try to find error with cholensky
hmm_est, loglikelihood_evolution = baum_welch(HMMjl, vectorized_data, atol=1e-10, max_iterations=1000);